### Trabalho realizado pelo grupo G18

- #### José Silva A100105
- #### Alexandra Calafate A100060


# TP2 - Problema 2

Considere o problema descrito no documento +Lógica Computacional: Multiplicação de Inteiros . Nesse documento usa-se um “Control Flow Automaton” como modelo do programa imperativo que calcula a multiplicação de inteiros positivos representados por vetores de bits.

Pretende-se

1.  Construir um SFOTS, usando BitVec’s de tamanho $n$ , que descreva o comportamento deste autómato; para isso identifique e codifique em `Z3` ou `pySMT`, as variáveis do modelo, o estado inicial , a relação de transição e o estado de erro.
2.  Usando $k$-indução verifique nesse SFOTS se a propriedade $\;(x*y + z = a*b)\;$ é um invariante do seu comportamento.
3.  Usando $k$-indução no FOTS acima e adicionando ao estado inicial a condição, $\,(a < 2^{n/2})\land(b < 2^{n/2})\,$, verifique a segurança do programa; nomeadamente prove que, com tal estado inicial, o estado de erro nunca é acessível.


In [ ]:
from z3 import *

# Define o tamanho dos BitVecs
n = 16  

# Definição das variáveis do modelo
a = BitVec('a', n)  # Primeiro fator da multiplicação
b = BitVec('b', n)  # Segundo fator da multiplicação
x = BitVec('x', n)  # Acumulador de multiplicações parciais
y = BitVec('y', n)  # Contador decremental
z = BitVec('z', n)  # Resultado parcial

# 1. Estado Inicial
# Define as condições iniciais do sistema
initial_state = And(x == a,    # x começa como a
                   y == b,     # y começa como b
                   z == 0,     # z começa como 0
                   a >= 0,     # a é não-negativo
                   b >= 0)     # b é não-negativo

# 2. Invariante
# Define a propriedade que deve ser mantida em todos os estados
invariant = (x * y + z == a * b)

# 3. Verificação da Indução Base
s = Solver()
s.add(initial_state)      
s.add(Not(invariant))     
print("Indução base:", s.check())

# 4. Relação de Transição
def transition(state):
    """Define as transições possíveis do sistema."""
    x, y, z = state['x'], state['y'], state['z']
    
    # Verifica se y é par
    even_y = y & 1 == 0
    
    # Calcula os shifts
    x_shifted = x << 1   # x * 2
    y_shifted = LShR(y, 1)  # y / 2
    
    # Define as transições possíveis
    next_state_if_even = And(x == x_shifted, y == y_shifted, z == z)
    next_state_if_odd = And(x == x, y == y - 1, z == z + x)
    
    return Or(
        And(even_y, next_state_if_even),
        And(Not(even_y), next_state_if_odd)
    )

# 5. K-indução
k = 2  # Profundidade da indução
for i in range(k):
    s.push()
    s.add(transition({'x': x, 'y': y, 'z': z}), invariant)

    if s.check() == unsat:
        print(f"Invariante é válido até o passo {i + 1}")
    else:
        print(f"Invariante falha no passo {i + 1}")
    s.pop()

# 6. Verificação de Segurança
s.push()
safe_initial_condition = And(a < 2**(n//2), b < 2**(n//2))
s.add(safe_initial_condition, initial_state)

if s.check() == unsat:
    print("Prova de segurança: Estado de erro é inacessível")
else:
    print("Prova de segurança falhou: Estado de erro pode ser acessível")
s.pop()